# `Flight tickets price prediction`

## Import Packages

In [262]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='whitegrid')
import warnings 
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import plotly.express as px
import plotly.figure_factory as ff
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder , MinMaxScaler , RobustScaler
from category_encoders import BinaryEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import r2_score,mean_squared_error,make_scorer
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import GridSearchCV
import joblib as jb

## Read Data

In [199]:
df = pd.read_excel('Data_Train.xlsx',na_values=['nan'])
df.sample(20)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
1149,Multiple carriers,27/03/2019,Delhi,Cochin,DEL → HYD → MAA → COK,07:10,21:20,14h 10m,2 stops,No info,6197
8142,Jet Airways,6/04/2019,Banglore,Delhi,BLR → DEL,18:55,22:00,3h 5m,non-stop,In-flight meal not included,4544
5754,SpiceJet,6/05/2019,Chennai,Kolkata,MAA → CCU,17:45,20:05,2h 20m,non-stop,No check-in baggage included,3543
1786,Air India,15/06/2019,Delhi,Cochin,DEL → AMD → BOM → COK,19:45,19:15 16 Jun,23h 30m,2 stops,No info,9653
663,SpiceJet,9/06/2019,Chennai,Kolkata,MAA → CCU,08:20,10:35,2h 15m,non-stop,No info,3597
9025,SpiceJet,27/05/2019,Banglore,Delhi,BLR → DEL,09:30,12:20,2h 50m,non-stop,No check-in baggage included,3573
7498,Jet Airways,6/05/2019,Kolkata,Banglore,CCU → DEL → BLR,20:25,23:35 07 May,27h 10m,1 stop,No info,10953
10018,Jet Airways,9/06/2019,Delhi,Cochin,DEL → BOM → COK,19:45,19:00 10 Jun,23h 15m,1 stop,In-flight meal not included,10577
5568,Jet Airways,6/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
9816,Jet Airways,15/06/2019,Delhi,Cochin,DEL → BOM → COK,11:30,19:00,7h 30m,1 stop,No info,14714


# 1. Data cleaning 


### 1.1 Investegation 

In [200]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [201]:
df.describe()

,Price
count,10683.000000
mean,9087.064121
std,4611.359167
min,1759.000000
25%,5277.000000
50%,8372.000000
75%,12373.000000
max,79512.000000


In [202]:
# Check Duplicated records
df.duplicated().sum()

220

In [203]:
# check null values in data
df.isna().sum()/df.shape[0]

Airline            0.000000
Date_of_Journey    0.000000
Source             0.000000
Destination        0.000000
Route              0.000094
Dep_Time           0.000000
Arrival_Time       0.000000
Duration           0.000000
Total_Stops        0.000094
Additional_Info    0.000000
Price              0.000000
dtype: float64

In [204]:
# Check Count of unique values in each feature
for col in df.columns:
    print ((col.ljust(20)), '=>' , (df[col].nunique()))

Airline              => 12
Date_of_Journey      => 44
Source               => 5
Destination          => 6
Route                => 128
Dep_Time             => 222
Arrival_Time         => 1343
Duration             => 368
Total_Stops          => 5
Additional_Info      => 10
Price                => 1870


In [205]:
## check the unique values of each feature 
for col in df.columns:
    print (col)
    print(df[col].unique())
    print(50*'*')

Airline
['IndiGo' 'Air India' 'Jet Airways' 'SpiceJet' 'Multiple carriers' 'GoAir'
 'Vistara' 'Air Asia' 'Vistara Premium economy' 'Jet Airways Business'
 'Multiple carriers Premium economy' 'Trujet']
**************************************************
Date_of_Journey
['24/03/2019' '1/05/2019' '9/06/2019' '12/05/2019' '01/03/2019'
 '24/06/2019' '12/03/2019' '27/05/2019' '1/06/2019' '18/04/2019'
 '9/05/2019' '24/04/2019' '3/03/2019' '15/04/2019' '12/06/2019'
 '6/03/2019' '21/03/2019' '3/04/2019' '6/05/2019' '15/05/2019'
 '18/06/2019' '15/06/2019' '6/04/2019' '18/05/2019' '27/06/2019'
 '21/05/2019' '06/03/2019' '3/06/2019' '15/03/2019' '3/05/2019'
 '9/03/2019' '6/06/2019' '24/05/2019' '09/03/2019' '1/04/2019'
 '21/04/2019' '21/06/2019' '27/03/2019' '18/03/2019' '12/04/2019'
 '9/04/2019' '1/03/2019' '03/03/2019' '27/04/2019']
**************************************************
Source
['Banglore' 'Kolkata' 'Delhi' 'Chennai' 'Mumbai']
**************************************************
Destina

#### Notes 
- There is 1 row has Nan Values. 
- Data has duplicated records
- change Date_of_Journey column type to datetime 

###  1.2 Working with data issues 

> Dropping duplicated Rows 

In [206]:
# Remove duplicated rows
df.drop_duplicates(inplace=True)

> Drop Null Row from data


In [207]:
# Remove Null Row from data
df.dropna(inplace=True)

In [208]:
# Reset Index after removing rows
df.reset_index(inplace=True)

In [209]:
# Check data after filteration
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10462 entries, 0 to 10461
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   index            10462 non-null  int64 
 1   Airline          10462 non-null  object
 2   Date_of_Journey  10462 non-null  object
 3   Source           10462 non-null  object
 4   Destination      10462 non-null  object
 5   Route            10462 non-null  object
 6   Dep_Time         10462 non-null  object
 7   Arrival_Time     10462 non-null  object
 8   Duration         10462 non-null  object
 9   Total_Stops      10462 non-null  object
 10  Additional_Info  10462 non-null  object
 11  Price            10462 non-null  int64 
dtypes: int64(2), object(10)
memory usage: 980.9+ KB


In [210]:
df.columns

Index(['index', 'Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info', 'Price'],
      dtype='object')

In [211]:
# Replace No info by No Info in Additional_Info column
df['Additional_Info'] = df['Additional_Info'].replace('No info', 'No Info')

In [212]:
# Extract Date from Arrival_Time column
df['Arrival_Date'] = df['Arrival_Time'].apply(lambda x : x.split()[1:] if len(str(x).split()) ==3 else 'None')
df['Arrival_Date'] = df['Arrival_Date'].apply(' '.join)

In [213]:
df

,index,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Arrival_Date
0,0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No Info,3897,22 Mar
1,1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No Info,7662,N o n e
2,2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No Info,13882,10 Jun
3,3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No Info,6218,N o n e
4,4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No Info,13302,N o n e
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10457,10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No Info,4107,N o n e
10458,10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No Info,4145,N o n e
10459,10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No Info,7229,N o n e
10460,10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No Info,12648,N o n e


In [214]:
# Convert the Date_of_Journey column to datetime data type
df['Date_of_Journey'] = pd.to_datetime(df['Date_of_Journey'], format='%d/%m/%Y')

# Convert the Arrival_Date column to datetime data type
df['Arrival_Date'] = pd.to_datetime(df['Arrival_Date'] + ' ' + df['Date_of_Journey'].dt.strftime('%Y'), format='%d %b %Y', errors='coerce').dt.strftime('%d/%m/%Y')

# Fill the missing dates in Arrival_Date column with corresponding dates from Date_of_Journey column
df['Arrival_Date'] = df['Arrival_Date'].fillna(df['Date_of_Journey'].dt.strftime('%d/%m/%Y'))

In [215]:
df['Ar_Time'] = df['Arrival_Time'].apply(lambda x : x.split()[0])

In [216]:
df.sample(20)

,index,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Arrival_Date,Ar_Time
6573,6652,IndiGo,2019-05-21,Kolkata,Banglore,CCU → HYD → BLR,15:15,20:05,4h 50m,1 stop,No Info,5170,21/05/2019,20:05
10100,10305,SpiceJet,2019-03-18,Chennai,Kolkata,MAA → CCU,08:20,10:35,2h 15m,non-stop,No check-in baggage included,3807,18/03/2019,10:35
2159,2171,Air India,2019-06-09,Kolkata,Banglore,CCU → BBI → BOM → BLR,12:00,11:10 10 Jun,23h 10m,2 stops,No Info,10151,10/06/2019,11:10
1447,1451,IndiGo,2019-03-21,Delhi,Cochin,DEL → BOM → COK,10:45,01:35 22 Mar,14h 50m,1 stop,No Info,7318,22/03/2019,01:35
7645,7750,Air India,2019-03-24,Kolkata,Banglore,CCU → GAU → DEL → BLR,16:45,23:15 25 Mar,30h 30m,2 stops,No Info,13885,25/03/2019,23:15
5696,5761,Jet Airways,2019-03-09,Delhi,Cochin,DEL → COK,11:00,14:15,3h 15m,non-stop,No Info,9564,09/03/2019,14:15
8325,8450,IndiGo,2019-03-09,Kolkata,Banglore,CCU → BLR,21:25,00:05 10 Mar,2h 40m,non-stop,No Info,4148,10/03/2019,00:05
8223,8345,Jet Airways,2019-05-18,Delhi,Cochin,DEL → JAI → BOM → COK,09:40,12:35 19 May,26h 55m,2 stops,In-flight meal not included,13029,19/05/2019,12:35
9935,10128,Jet Airways,2019-05-06,Kolkata,Banglore,CCU → BOM → BLR,21:10,08:15 07 May,11h 5m,1 stop,In-flight meal not included,7757,07/05/2019,08:15
6642,6723,Jet Airways,2019-05-18,Delhi,Cochin,DEL → BOM → COK,09:00,12:35 19 May,27h 35m,1 stop,In-flight meal not included,12373,19/05/2019,12:35


In [217]:
# Convert the Date_of_Journey column to string data type
df['Date_of_Journey'] = df['Date_of_Journey'].astype(str)

In [218]:
# Merge the Date_of_Journey and Dep_Time columns into a single datetime column Dep_Date
df['Dep_Date'] = pd.to_datetime(df['Date_of_Journey'] + ' ' + df['Dep_Time'])

# Merge the Arrival_Date and Arrival time columns into a single datetime column Arr_Date
df['Arr_Date'] = pd.to_datetime(df['Arrival_Date'] + ' ' + df['Ar_Time'], format='%d/%m/%Y %H:%M', errors='coerce')

In [219]:
df.sample(20)

,index,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Arrival_Date,Ar_Time,Dep_Date,Arr_Date
5328,5388,IndiGo,2019-03-01,Mumbai,Hyderabad,BOM → HYD,02:30,04:00,1h 30m,non-stop,No Info,16000,01/03/2019,04:00,2019-03-01 02:30:00,2019-03-01 04:00:00
9019,9172,Vistara,2019-04-09,Banglore,Delhi,BLR → DEL,07:00,09:40,2h 40m,non-stop,No Info,5403,09/04/2019,09:40,2019-04-09 07:00:00,2019-04-09 09:40:00
6718,6802,Vistara,2019-04-06,Banglore,Delhi,BLR → DEL,19:30,22:15,2h 45m,non-stop,No Info,5613,06/04/2019,22:15,2019-04-06 19:30:00,2019-04-06 22:15:00
8708,8847,IndiGo,2019-06-15,Chennai,Kolkata,MAA → CCU,05:15,07:40,2h 25m,non-stop,No Info,3540,15/06/2019,07:40,2019-06-15 05:15:00,2019-06-15 07:40:00
8516,8652,Multiple carriers,2019-06-24,Delhi,Cochin,DEL → HYD → COK,13:15,22:30,9h 15m,1 stop,No Info,8018,24/06/2019,22:30,2019-06-24 13:15:00,2019-06-24 22:30:00
6581,6661,Multiple carriers,2019-03-09,Delhi,Cochin,DEL → BOM → COK,04:45,19:15,14h 30m,1 stop,No Info,11249,09/03/2019,19:15,2019-03-09 04:45:00,2019-03-09 19:15:00
3893,3930,SpiceJet,2019-03-03,Banglore,New Delhi,BLR → HYD → DEL,15:35,23:40,8h 5m,1 stop,No Info,6315,03/03/2019,23:40,2019-03-03 15:35:00,2019-03-03 23:40:00
4028,4067,Jet Airways,2019-03-27,Delhi,Cochin,DEL → BOM → COK,11:30,04:25 28 Mar,16h 55m,1 stop,No Info,12242,28/03/2019,04:25,2019-03-27 11:30:00,2019-03-28 04:25:00
4583,4630,SpiceJet,2019-06-03,Banglore,Delhi,BLR → DEL,21:10,00:05 04 Jun,2h 55m,non-stop,No Info,3311,04/06/2019,00:05,2019-06-03 21:10:00,2019-06-04 00:05:00
2458,2470,Jet Airways,2019-05-21,Kolkata,Banglore,CCU → DEL → BLR,09:35,19:10,9h 35m,1 stop,In-flight meal not included,12384,21/05/2019,19:10,2019-05-21 09:35:00,2019-05-21 19:10:00


In [220]:
# Check Types of new columns 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10462 entries, 0 to 10461
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   index            10462 non-null  int64         
 1   Airline          10462 non-null  object        
 2   Date_of_Journey  10462 non-null  object        
 3   Source           10462 non-null  object        
 4   Destination      10462 non-null  object        
 5   Route            10462 non-null  object        
 6   Dep_Time         10462 non-null  object        
 7   Arrival_Time     10462 non-null  object        
 8   Duration         10462 non-null  object        
 9   Total_Stops      10462 non-null  object        
 10  Additional_Info  10462 non-null  object        
 11  Price            10462 non-null  int64         
 12  Arrival_Date     10462 non-null  object        
 13  Ar_Time          10462 non-null  object        
 14  Dep_Date         10462 non-null  datet

In [221]:
# Calculate the time difference between the two columns
df['Accurate_difference'] = df['Arr_Date'] - df['Dep_Date']

# Convert the duration column to timedelta data type
df['Duration'] = pd.to_timedelta(df['Duration'])

# Find the rows where duration is not equal to Accurate difference
df_diff = df[df['Duration'] != df['Accurate_difference']]

# Print the resulting DataFrame
print(df_diff)

       index      Airline Date_of_Journey    Source Destination  \
0          0       IndiGo      2019-03-24  Banglore   New Delhi   
21        21    Air India      2019-03-21  Banglore   New Delhi   
52        52       IndiGo      2019-03-24  Banglore   New Delhi   
71        71  Jet Airways      2019-03-09  Banglore   New Delhi   
110      110  Jet Airways      2019-03-18  Banglore   New Delhi   
...      ...          ...             ...       ...         ...   
10233  10441  Jet Airways      2019-03-18  Banglore   New Delhi   
10279  10489       IndiGo      2019-03-15   Chennai     Kolkata   
10374  10590     Air Asia      2019-03-24  Banglore   New Delhi   
10419  10638  Jet Airways      2019-03-21  Banglore   New Delhi   
10441  10660      Vistara      2019-03-21  Banglore   New Delhi   

                 Route Dep_Time  Arrival_Time        Duration Total_Stops  \
0            BLR → DEL    22:20  01:10 22 Mar 0 days 02:50:00    non-stop   
21     BLR → COK → DEL    22:00  13:20 19

 > Because 144 Rows are only about 1% of my data so i will drop them

In [222]:
# Drop the rows where duration is not equal to Accurate difference
df = df[df['Duration'] == df['Accurate_difference']]

In [223]:
# Check after Filteration

# Find the rows where duration is not equal to Accurate difference
df_diff = df[df['Duration'] != df['Accurate_difference']]

# Print the resulting DataFrame
print(df_diff)

Empty DataFrame
Columns: [index, Airline, Date_of_Journey, Source, Destination, Route, Dep_Time, Arrival_Time, Duration, Total_Stops, Additional_Info, Price, Arrival_Date, Ar_Time, Dep_Date, Arr_Date, Accurate_difference]
Index: []


In [224]:
df

,index,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Arrival_Date,Ar_Time,Dep_Date,Arr_Date,Accurate_difference
1,1,Air India,2019-05-01,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,0 days 07:25:00,2 stops,No Info,7662,01/05/2019,13:15,2019-05-01 05:50:00,2019-05-01 13:15:00,0 days 07:25:00
2,2,Jet Airways,2019-06-09,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,0 days 19:00:00,2 stops,No Info,13882,10/06/2019,04:25,2019-06-09 09:25:00,2019-06-10 04:25:00,0 days 19:00:00
3,3,IndiGo,2019-05-12,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,0 days 05:25:00,1 stop,No Info,6218,12/05/2019,23:30,2019-05-12 18:05:00,2019-05-12 23:30:00,0 days 05:25:00
4,4,IndiGo,2019-03-01,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,0 days 04:45:00,1 stop,No Info,13302,01/03/2019,21:35,2019-03-01 16:50:00,2019-03-01 21:35:00,0 days 04:45:00
5,5,SpiceJet,2019-06-24,Kolkata,Banglore,CCU → BLR,09:00,11:25,0 days 02:25:00,non-stop,No Info,3873,24/06/2019,11:25,2019-06-24 09:00:00,2019-06-24 11:25:00,0 days 02:25:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10457,10678,Air Asia,2019-04-09,Kolkata,Banglore,CCU → BLR,19:55,22:25,0 days 02:30:00,non-stop,No Info,4107,09/04/2019,22:25,2019-04-09 19:55:00,2019-04-09 22:25:00,0 days 02:30:00
10458,10679,Air India,2019-04-27,Kolkata,Banglore,CCU → BLR,20:45,23:20,0 days 02:35:00,non-stop,No Info,4145,27/04/2019,23:20,2019-04-27 20:45:00,2019-04-27 23:20:00,0 days 02:35:00
10459,10680,Jet Airways,2019-04-27,Banglore,Delhi,BLR → DEL,08:20,11:20,0 days 03:00:00,non-stop,No Info,7229,27/04/2019,11:20,2019-04-27 08:20:00,2019-04-27 11:20:00,0 days 03:00:00
10460,10681,Vistara,2019-03-01,Banglore,New Delhi,BLR → DEL,11:30,14:10,0 days 02:40:00,non-stop,No Info,12648,01/03/2019,14:10,2019-03-01 11:30:00,2019-03-01 14:10:00,0 days 02:40:00


In [225]:
df['Total_Stops'].unique()

array(['2 stops', '1 stop', 'non-stop', '3 stops', '4 stops'],
      dtype=object)

In [226]:
# Make anew column to map stops count and check if it's right or not
Stops_Check = {'non-stop':0, '1 stop':1, '2 stops':2, '3 stops':3, '4 stops':4}
Total_Stops = {'non-stop':4, '1 stop':3, '2 stops':2, '3 stops':1, '4 stops':0}

# apply using map
df['Stops_Check'] = df['Total_Stops'].map(Stops_Check)

In [227]:
# Count the number of stops in the routes column
df['count_stops'] = df['Route'].str.count('→') -1

# Filter the rows where count stops doesn't equal total stops
df_filtered = df[df['count_stops'] != df['Stops_Check']]

# Print the resulting DataFrame
print(df_filtered)

Empty DataFrame
Columns: [index, Airline, Date_of_Journey, Source, Destination, Route, Dep_Time, Arrival_Time, Duration, Total_Stops, Additional_Info, Price, Arrival_Date, Ar_Time, Dep_Date, Arr_Date, Accurate_difference, Stops_Check, count_stops]
Index: []


> All Total stops are right 

In [228]:
# Reset Index of Data records
df.reset_index(drop = True,inplace = True)

In [229]:
df

,index,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Arrival_Date,Ar_Time,Dep_Date,Arr_Date,Accurate_difference,Stops_Check,count_stops
0,1,Air India,2019-05-01,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,0 days 07:25:00,2 stops,No Info,7662,01/05/2019,13:15,2019-05-01 05:50:00,2019-05-01 13:15:00,0 days 07:25:00,2,2
1,2,Jet Airways,2019-06-09,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,0 days 19:00:00,2 stops,No Info,13882,10/06/2019,04:25,2019-06-09 09:25:00,2019-06-10 04:25:00,0 days 19:00:00,2,2
2,3,IndiGo,2019-05-12,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,0 days 05:25:00,1 stop,No Info,6218,12/05/2019,23:30,2019-05-12 18:05:00,2019-05-12 23:30:00,0 days 05:25:00,1,1
3,4,IndiGo,2019-03-01,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,0 days 04:45:00,1 stop,No Info,13302,01/03/2019,21:35,2019-03-01 16:50:00,2019-03-01 21:35:00,0 days 04:45:00,1,1
4,5,SpiceJet,2019-06-24,Kolkata,Banglore,CCU → BLR,09:00,11:25,0 days 02:25:00,non-stop,No Info,3873,24/06/2019,11:25,2019-06-24 09:00:00,2019-06-24 11:25:00,0 days 02:25:00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10313,10678,Air Asia,2019-04-09,Kolkata,Banglore,CCU → BLR,19:55,22:25,0 days 02:30:00,non-stop,No Info,4107,09/04/2019,22:25,2019-04-09 19:55:00,2019-04-09 22:25:00,0 days 02:30:00,0,0
10314,10679,Air India,2019-04-27,Kolkata,Banglore,CCU → BLR,20:45,23:20,0 days 02:35:00,non-stop,No Info,4145,27/04/2019,23:20,2019-04-27 20:45:00,2019-04-27 23:20:00,0 days 02:35:00,0,0
10315,10680,Jet Airways,2019-04-27,Banglore,Delhi,BLR → DEL,08:20,11:20,0 days 03:00:00,non-stop,No Info,7229,27/04/2019,11:20,2019-04-27 08:20:00,2019-04-27 11:20:00,0 days 03:00:00,0,0
10316,10681,Vistara,2019-03-01,Banglore,New Delhi,BLR → DEL,11:30,14:10,0 days 02:40:00,non-stop,No Info,12648,01/03/2019,14:10,2019-03-01 11:30:00,2019-03-01 14:10:00,0 days 02:40:00,0,0


# 2.Business Questions

## What is the most airline company used ?

In [230]:
p=px.histogram(df,x='Airline')
p.show()

## what is avarage ticket price for each total stop?

In [231]:
df.groupby("Total_Stops")["Price"].mean().sort_values(ascending = False).head(10)

Total_Stops
4 stops     17686.000000
3 stops     13282.166667
2 stops     12776.147604
1 stop      10578.214221
non-stop     5016.152866
Name: Price, dtype: float64

In [252]:
df.groupby("Total_Stops")["Price"].median().sort_values(ascending = False).head(10)

Total_Stops
4    17686.0
3    13007.0
2    12768.5
1    10262.0
0     4659.0
Name: Price, dtype: float64

## what is the Furthest route takes time ?

In [232]:
df.groupby("Route")["Duration"].mean().sort_values(ascending = False).head(10)

Route
CCU → BBI → IXR → DEL → BLR                   1 days 11:15:00
DEL → RPR → NAG → BOM → COK                   1 days 10:54:40
CCU → IXB → DEL → BLR                         1 days 09:20:00
CCU → IXZ → MAA → BLR                         1 days 08:55:00
BOM → JAI → DEL → HYD                         1 days 07:50:00
BLR → BOM → UDR → DEL                         1 days 07:40:00
BLR → BOM → IDR → GWL → DEL                   1 days 06:00:00
BOM → BHO → DEL → HYD               1 days 05:47:51.428571428
BOM → JDH → JAI → DEL → HYD                   1 days 05:35:00
BLR → CCU → BBI → HYD → VGA → DEL             1 days 05:30:00
Name: Duration, dtype: timedelta64[ns]

## Check Correlation 

In [233]:
px.imshow(df.corr().round(2),text_auto=True)

In [234]:
df

,index,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Arrival_Date,Ar_Time,Dep_Date,Arr_Date,Accurate_difference,Stops_Check,count_stops
0,1,Air India,2019-05-01,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,0 days 07:25:00,2 stops,No Info,7662,01/05/2019,13:15,2019-05-01 05:50:00,2019-05-01 13:15:00,0 days 07:25:00,2,2
1,2,Jet Airways,2019-06-09,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,0 days 19:00:00,2 stops,No Info,13882,10/06/2019,04:25,2019-06-09 09:25:00,2019-06-10 04:25:00,0 days 19:00:00,2,2
2,3,IndiGo,2019-05-12,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,0 days 05:25:00,1 stop,No Info,6218,12/05/2019,23:30,2019-05-12 18:05:00,2019-05-12 23:30:00,0 days 05:25:00,1,1
3,4,IndiGo,2019-03-01,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,0 days 04:45:00,1 stop,No Info,13302,01/03/2019,21:35,2019-03-01 16:50:00,2019-03-01 21:35:00,0 days 04:45:00,1,1
4,5,SpiceJet,2019-06-24,Kolkata,Banglore,CCU → BLR,09:00,11:25,0 days 02:25:00,non-stop,No Info,3873,24/06/2019,11:25,2019-06-24 09:00:00,2019-06-24 11:25:00,0 days 02:25:00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10313,10678,Air Asia,2019-04-09,Kolkata,Banglore,CCU → BLR,19:55,22:25,0 days 02:30:00,non-stop,No Info,4107,09/04/2019,22:25,2019-04-09 19:55:00,2019-04-09 22:25:00,0 days 02:30:00,0,0
10314,10679,Air India,2019-04-27,Kolkata,Banglore,CCU → BLR,20:45,23:20,0 days 02:35:00,non-stop,No Info,4145,27/04/2019,23:20,2019-04-27 20:45:00,2019-04-27 23:20:00,0 days 02:35:00,0,0
10315,10680,Jet Airways,2019-04-27,Banglore,Delhi,BLR → DEL,08:20,11:20,0 days 03:00:00,non-stop,No Info,7229,27/04/2019,11:20,2019-04-27 08:20:00,2019-04-27 11:20:00,0 days 03:00:00,0,0
10316,10681,Vistara,2019-03-01,Banglore,New Delhi,BLR → DEL,11:30,14:10,0 days 02:40:00,non-stop,No Info,12648,01/03/2019,14:10,2019-03-01 11:30:00,2019-03-01 14:10:00,0 days 02:40:00,0,0


# 3. Feature Engineering

In [235]:
# Extract the day, month, year, hour, and minute components into separate columns
df['Dep_Day'] = df['Dep_Date'].dt.day
df['Dep_Month'] = df['Dep_Date'].dt.month
df['Dep_Hour'] = df['Dep_Date'].dt.hour
df['Dep_Minute'] = df['Dep_Date'].dt.minute

df['Arrival_Day'] = df['Arr_Date'].dt.day
df['Arrival_Month'] = df['Arr_Date'].dt.month
df['Arrival_Hour'] = df['Arr_Date'].dt.hour
df['Arrival_Minute'] = df['Arr_Date'].dt.minute

In [238]:
# Check dataFrame after Extracting
df.T

,0,1,2,3,4,5,6,7,8,9,...,10308,10309,10310,10311,10312,10313,10314,10315,10316,10317
index,1,2,3,4,5,6,7,8,9,10,...,10671,10674,10675,10676,10677,10678,10679,10680,10681,10682
Airline,Air India,Jet Airways,IndiGo,IndiGo,SpiceJet,Jet Airways,Jet Airways,Jet Airways,Multiple carriers,Air India,...,Air India,Jet Airways,Air India,Multiple carriers,SpiceJet,Air Asia,Air India,Jet Airways,Vistara,Air India
Date_of_Journey,2019-05-01,2019-06-09,2019-05-12,2019-03-01,2019-06-24,2019-03-12,2019-03-01,2019-03-12,2019-05-27,2019-06-01,...,2019-06-06,2019-03-12,2019-06-09,2019-05-01,2019-05-21,2019-04-09,2019-04-27,2019-04-27,2019-03-01,2019-05-09
Source,Kolkata,Delhi,Kolkata,Banglore,Kolkata,Banglore,Banglore,Banglore,Delhi,Delhi,...,Mumbai,Banglore,Mumbai,Delhi,Banglore,Kolkata,Kolkata,Banglore,Banglore,Delhi
Destination,Banglore,Cochin,Banglore,New Delhi,Banglore,New Delhi,New Delhi,New Delhi,Cochin,Cochin,...,Hyderabad,New Delhi,Hyderabad,Cochin,Delhi,Banglore,Banglore,Delhi,New Delhi,Cochin
Route,CCU → IXR → BBI → BLR,DEL → LKO → BOM → COK,CCU → NAG → BLR,BLR → NAG → DEL,CCU → BLR,BLR → BOM → DEL,BLR → BOM → DEL,BLR → BOM → DEL,DEL → BOM → COK,DEL → BLR → COK,...,BOM → HYD,BLR → BOM → DEL,BOM → HYD,DEL → BOM → COK,BLR → DEL,CCU → BLR,CCU → BLR,BLR → DEL,BLR → DEL,DEL → GOI → BOM → COK
Dep_Time,05:50,09:25,18:05,16:50,09:00,18:55,08:00,08:55,11:25,09:45,...,21:05,20:35,06:20,10:20,05:55,19:55,20:45,08:20,11:30,10:55
Arrival_Time,13:15,04:25 10 Jun,23:30,21:35,11:25,10:25 13 Mar,05:05 02 Mar,10:25 13 Mar,19:15,23:00,...,22:25,21:20 13 Mar,07:40,19:00,08:35,22:25,23:20,11:20,14:10,19:15
Duration,0 days 07:25:00,0 days 19:00:00,0 days 05:25:00,0 days 04:45:00,0 days 02:25:00,0 days 15:30:00,0 days 21:05:00,1 days 01:30:00,0 days 07:50:00,0 days 13:15:00,...,0 days 01:20:00,1 days 00:45:00,0 days 01:20:00,0 days 08:40:00,0 days 02:40:00,0 days 02:30:00,0 days 02:35:00,0 days 03:00:00,0 days 02:40:00,0 days 08:20:00
Total_Stops,2 stops,2 stops,1 stop,1 stop,non-stop,1 stop,1 stop,1 stop,1 stop,1 stop,...,non-stop,1 stop,non-stop,1 stop,non-stop,non-stop,non-stop,non-stop,non-stop,2 stops


In [240]:
# convert duration to minutes
df['Duration'] = pd.to_timedelta(df['Duration']).dt.total_seconds() / 60

# update the duration column with the calculated values
df.loc[:, 'Duration'] = df['Duration'].astype(int)

In [241]:
# Check dataFrame after Replacing
df.T

,0,1,2,3,4,5,6,7,8,9,...,10308,10309,10310,10311,10312,10313,10314,10315,10316,10317
index,1,2,3,4,5,6,7,8,9,10,...,10671,10674,10675,10676,10677,10678,10679,10680,10681,10682
Airline,Air India,Jet Airways,IndiGo,IndiGo,SpiceJet,Jet Airways,Jet Airways,Jet Airways,Multiple carriers,Air India,...,Air India,Jet Airways,Air India,Multiple carriers,SpiceJet,Air Asia,Air India,Jet Airways,Vistara,Air India
Date_of_Journey,2019-05-01,2019-06-09,2019-05-12,2019-03-01,2019-06-24,2019-03-12,2019-03-01,2019-03-12,2019-05-27,2019-06-01,...,2019-06-06,2019-03-12,2019-06-09,2019-05-01,2019-05-21,2019-04-09,2019-04-27,2019-04-27,2019-03-01,2019-05-09
Source,Kolkata,Delhi,Kolkata,Banglore,Kolkata,Banglore,Banglore,Banglore,Delhi,Delhi,...,Mumbai,Banglore,Mumbai,Delhi,Banglore,Kolkata,Kolkata,Banglore,Banglore,Delhi
Destination,Banglore,Cochin,Banglore,New Delhi,Banglore,New Delhi,New Delhi,New Delhi,Cochin,Cochin,...,Hyderabad,New Delhi,Hyderabad,Cochin,Delhi,Banglore,Banglore,Delhi,New Delhi,Cochin
Route,CCU → IXR → BBI → BLR,DEL → LKO → BOM → COK,CCU → NAG → BLR,BLR → NAG → DEL,CCU → BLR,BLR → BOM → DEL,BLR → BOM → DEL,BLR → BOM → DEL,DEL → BOM → COK,DEL → BLR → COK,...,BOM → HYD,BLR → BOM → DEL,BOM → HYD,DEL → BOM → COK,BLR → DEL,CCU → BLR,CCU → BLR,BLR → DEL,BLR → DEL,DEL → GOI → BOM → COK
Dep_Time,05:50,09:25,18:05,16:50,09:00,18:55,08:00,08:55,11:25,09:45,...,21:05,20:35,06:20,10:20,05:55,19:55,20:45,08:20,11:30,10:55
Arrival_Time,13:15,04:25 10 Jun,23:30,21:35,11:25,10:25 13 Mar,05:05 02 Mar,10:25 13 Mar,19:15,23:00,...,22:25,21:20 13 Mar,07:40,19:00,08:35,22:25,23:20,11:20,14:10,19:15
Duration,445,1140,325,285,145,930,1265,1530,470,795,...,80,1485,80,520,160,150,155,180,160,500
Total_Stops,2 stops,2 stops,1 stop,1 stop,non-stop,1 stop,1 stop,1 stop,1 stop,1 stop,...,non-stop,1 stop,non-stop,1 stop,non-stop,non-stop,non-stop,non-stop,non-stop,2 stops


In [243]:
# Drop useless Columns
df.drop(['index','Date_of_Journey','Route','Dep_Time','Arrival_Time','Total_Stops','Arrival_Date','Ar_Time',
       'Dep_Date', 'Arr_Date','Accurate_difference','Stops_Check'],axis=1,inplace=True)

In [248]:
# Rename count_stops to Total_Stops
df = df.rename(columns={'count_stops': 'Total_Stops'})

In [249]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,10308,10309,10310,10311,10312,10313,10314,10315,10316,10317
Airline,Air India,Jet Airways,IndiGo,IndiGo,SpiceJet,Jet Airways,Jet Airways,Jet Airways,Multiple carriers,Air India,...,Air India,Jet Airways,Air India,Multiple carriers,SpiceJet,Air Asia,Air India,Jet Airways,Vistara,Air India
Source,Kolkata,Delhi,Kolkata,Banglore,Kolkata,Banglore,Banglore,Banglore,Delhi,Delhi,...,Mumbai,Banglore,Mumbai,Delhi,Banglore,Kolkata,Kolkata,Banglore,Banglore,Delhi
Destination,Banglore,Cochin,Banglore,New Delhi,Banglore,New Delhi,New Delhi,New Delhi,Cochin,Cochin,...,Hyderabad,New Delhi,Hyderabad,Cochin,Delhi,Banglore,Banglore,Delhi,New Delhi,Cochin
Duration,445,1140,325,285,145,930,1265,1530,470,795,...,80,1485,80,520,160,150,155,180,160,500
Additional_Info,No Info,No Info,No Info,No Info,No Info,In-flight meal not included,No Info,In-flight meal not included,No Info,No Info,...,No Info,In-flight meal not included,No Info,No Info,No check-in baggage included,No Info,No Info,No Info,No Info,No Info
Price,7662,13882,6218,13302,3873,11087,22270,11087,8625,8907,...,3100,11087,3100,9794,3257,4107,4145,7229,12648,11753
Total_Stops,2,2,1,1,0,1,1,1,1,1,...,0,1,0,1,0,0,0,0,0,2
Dep_Day,1,9,12,1,24,12,1,12,27,1,...,6,12,9,1,21,9,27,27,1,9
Dep_Month,5,6,5,3,6,3,3,3,5,6,...,6,3,6,5,5,4,4,4,3,5
Dep_Hour,5,9,18,16,9,18,8,8,11,9,...,21,20,6,10,5,19,20,8,11,10


In [247]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10318 entries, 0 to 10317
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10318 non-null  object
 1   Source           10318 non-null  object
 2   Destination      10318 non-null  object
 3   Duration         10318 non-null  int32 
 4   Additional_Info  10318 non-null  object
 5   Price            10318 non-null  int64 
 6   count_stops      10318 non-null  int64 
 7   Dep_Day          10318 non-null  int64 
 8   Dep_Month        10318 non-null  int64 
 9   Dep_Hour         10318 non-null  int64 
 10  Dep_Minute       10318 non-null  int64 
 11  Arrival_Day      10318 non-null  int64 
 12  Arrival_Month    10318 non-null  int64 
 13  Arrival_Hour     10318 non-null  int64 
 14  Arrival_Minute   10318 non-null  int64 
dtypes: int32(1), int64(10), object(4)
memory usage: 1.1+ MB


In [264]:
# Creat list contains categorical Columns
Category_Cols=['Airline','Source','Destination','Additional_Info']

## 4. Handling Outliers

In [251]:
# Check outliers in Price column 
fig=px.box(df,x='Price')
fig.show()

In [255]:
# Check outliers in babies column 
df[df['Price'] > 23000]

,Airline,Source,Destination,Duration,Additional_Info,Price,Total_Stops,Dep_Day,Dep_Month,Dep_Hour,Dep_Minute,Arrival_Day,Arrival_Month,Arrival_Hour,Arrival_Minute
118,Air India,Delhi,Cochin,1215,No Info,27430,1,1,3,23,0,2,3,19,15
390,Multiple carriers,Delhi,Cochin,385,No Info,36983,1,1,3,12,50,1,3,19,15
480,Jet Airways,Banglore,New Delhi,380,No Info,26890,1,1,3,8,55,1,3,15,15
504,Jet Airways,Banglore,New Delhi,325,No Info,26890,1,1,3,7,0,1,3,12,25
591,Air India,Mumbai,Hyderabad,1775,No Info,25139,2,6,3,9,40,7,3,15,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10011,Jet Airways,Mumbai,Hyderabad,1260,No Info,24210,2,6,3,19,50,7,3,16,50
10022,Jet Airways Business,Banglore,New Delhi,280,Business class,57209,1,1,3,9,45,1,3,14,25
10040,Jet Airways,Banglore,New Delhi,1325,No Info,26890,1,1,3,7,0,2,3,5,5
10092,Jet Airways,Banglore,New Delhi,860,No Info,31825,1,1,3,18,40,2,3,9,0


In [256]:
# Drop outliers in Price column 
df.drop(df[df['Price'] > 23000].index, axis=0, inplace=True)

## Check Corr in data 

In [258]:
px.imshow(df.corr().round(2),text_auto=True)

## Data Encoding And Scaling & ML Models evaluation

In [260]:
## split it into x--> data,  y--> target
X = df.drop('Price', axis =1)
y = df['Price']

In [263]:
## Models 
models = [
    ('Linear Regression', LinearRegression()), 
    ('Ridge',Ridge()),
    ('Lasso',Lasso(alpha=0.001)),
    ('DTR', DecisionTreeRegressor()), 
    ('RFR', RandomForestRegressor()), 
    ('ADA', AdaBoostRegressor()), 
    ('GBR', GradientBoostingRegressor()), 
    ('XGBR', XGBRegressor())
]

In [269]:
# Define the numerical column
numerical_cols = X.select_dtypes(include='number').columns

# Initialize RFE feature selector
RFE_selector = RFE(XGBRegressor(), n_features_to_select=20) 

# Create the ColumnTransformer
preprocessor = ColumnTransformer(transformers=[('Cat',BinaryEncoder(), Category_Cols),
                                               ('num', StandardScaler(), numerical_cols)
                                              ],remainder='passthrough')


for model_name, model in models:
    # Print model name
    print(f"For {model_name}:")

    # Steps Creation
    steps = list()
    steps.append(('preprocessor', preprocessor))

    # Choose the feature selector based on your preference
    steps.append(('feature_selector', RFE_selector))  # RFE feature selection
    
    steps.append((model_name, model))

    # Create the pipeline
    pipeline = Pipeline(steps=steps)

    ## Scoring
    # Perform cross-validation
    # Example evaluation metrics
    cv_results = cross_validate(pipeline, X, y, scoring='r2', cv=5, return_train_score=True)  # X and y are your input data and targets



    # Access the results
    print("test :", cv_results['test_score'].mean())
    print("train:", cv_results['train_score'].mean())
    print('*' * 50)

For Linear Regression:
test : 0.5985494231487422
train: 0.6038828041182229
**************************************************
For Ridge:
test : 0.5987937569804096
train: 0.6040161175528523
**************************************************
For Lasso:
test : 0.5987943695073229
train: 0.6040183963389543
**************************************************
For DTR:
test : 0.8780614926986784
train: 0.9949199921610508
**************************************************
For RFR:
test : 0.9231710504966009
train: 0.9856320742049766
**************************************************
For ADA:
test : 0.596446066674537
train: 0.6042964372137966
**************************************************
For GBR:
test : 0.8144441271185062
train: 0.8259231027116465
**************************************************
For XGBR:
test : 0.9277427146735082
train: 0.973511547049567
**************************************************


In [270]:
selected_features= RFE_selector.fit_transform(preprocessor.fit_transform(X),y)
selected_df= pd.DataFrame(selected_features, columns=[f'Feature_{i}' for i in range(selected_features.shape[1])])
print(selected_df)

       Feature_0  Feature_1  Feature_2  Feature_3  Feature_4  Feature_5  \
0            0.0        0.0        0.0        1.0        0.0        0.0   
1            0.0        0.0        1.0        0.0        0.0        1.0   
2            0.0        0.0        1.0        1.0        0.0        0.0   
3            0.0        0.0        1.0        1.0        0.0        1.0   
4            0.0        1.0        0.0        0.0        0.0        0.0   
...          ...        ...        ...        ...        ...        ...   
10222        1.0        0.0        0.0        0.0        0.0        0.0   
10223        0.0        0.0        0.0        1.0        0.0        0.0   
10224        0.0        0.0        1.0        0.0        0.0        1.0   
10225        0.0        1.0        1.0        1.0        0.0        1.0   
10226        0.0        0.0        0.0        1.0        0.0        1.0   

       Feature_6  Feature_7  Feature_8  Feature_9  Feature_10  Feature_11  \
0            1.0      

In [271]:
preprocessor.get_feature_names_out()

array(['Cat__Airline_0', 'Cat__Airline_1', 'Cat__Airline_2',
       'Cat__Airline_3', 'Cat__Source_0', 'Cat__Source_1',
       'Cat__Source_2', 'Cat__Destination_0', 'Cat__Destination_1',
       'Cat__Destination_2', 'Cat__Additional_Info_0',
       'Cat__Additional_Info_1', 'Cat__Additional_Info_2',
       'num__Duration', 'num__Total_Stops', 'num__Dep_Day',
       'num__Dep_Month', 'num__Dep_Hour', 'num__Dep_Minute',
       'num__Arrival_Day', 'num__Arrival_Month', 'num__Arrival_Hour',
       'num__Arrival_Minute'], dtype=object)

In [272]:
RFE_selector.get_feature_names_out()

array(['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x11',
       'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x21', 'x22'],
      dtype=object)

## HyperParameter Tuning

In [273]:
# Define your parameter grid for GridSearchCV
param_grid = {
    'XGBR__learning_rate': [0.01, 0.1],
    'XGBR__max_depth': [3, 5, 7],
    'XGBR__n_estimators': [100, 500, 1000],
    'XGBR__min_child_weight': [1, 5, 10],
    'XGBR__gamma': [0, 0.1, 0.5]
}

In [274]:
steps=[]
steps.append(('preprocessor', preprocessor))
steps.append(('feature_selector', RFE_selector))
steps.append(('XGBR',XGBRegressor()))
pipeline=Pipeline(steps=steps)

In [275]:
# Create GridSearchCV instance
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1, return_train_score=True)

# Fit the pipeline with GridSearch to the data
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'XGBR__gamma': 0, 'XGBR__learning_rate': 0.1, 'XGBR__max_depth': 7, 'XGBR__min_child_weight': 10, 'XGBR__n_estimators': 500}
Best Score: 0.9314852267060976


In [288]:
# Define your parameter grid for GridSearchCV
param_grid = {
    'XGBR__learning_rate': [0.01, 0.1,.17],
    'XGBR__max_depth': [5, 7, 10],
    'XGBR__n_estimators': [100, 500, 1000],
    'XGBR__min_child_weight': [5, 10, 13],
    'XGBR__gamma': [0, 0.1, 0.5]
}

In [289]:
# Create GridSearchCV instance
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1, return_train_score=True)

# Fit the pipeline with GridSearch to the data
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'XGBR__gamma': 0, 'XGBR__learning_rate': 0.1, 'XGBR__max_depth': 7, 'XGBR__min_child_weight': 13, 'XGBR__n_estimators': 500}
Best Score: 0.9320918680507534


In [291]:
# Define your parameter grid for GridSearchCV
param_grid = {
    'XGBR__learning_rate': [0.01, 0.1,.17],
    'XGBR__max_depth': [5, 7, 10],
    'XGBR__n_estimators': [100, 500, 1000],
    'XGBR__min_child_weight': [10, 13, 16],
    'XGBR__gamma': [0, 0.1, 0.5]
}

In [293]:
# Create GridSearchCV instance
grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1, return_train_score=True)

# Fit the pipeline with GridSearch to the data
grid_search.fit(X, y)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'XGBR__gamma': 0, 'XGBR__learning_rate': 0.1, 'XGBR__max_depth': 7, 'XGBR__min_child_weight': 13, 'XGBR__n_estimators': 500}
Best Score: 0.9320918680507534


In [294]:
final_model=grid_search.best_estimator_
final_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Cat', BinaryEncoder(),
                                                  ['Airline', 'Source',
                                                   'Destination',
                                                   'Additional_Info']),
                                                 ('num', StandardScaler(),
                                                  Index(['Duration', 'Total_Stops', 'Dep_Day', 'Dep_Month', 'Dep_Hour',
       'Dep_Minute', 'Arrival_Day', 'Arrival_Month', 'Arrival_Hour',
       'Arrival_Minute'],
      dtype='object'))])),...
                              feature_types=None, gamma=0, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=0.1,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=7, max_leaves=None, min_child_weight=13,
                              missing=nan, monotone_constraints=None,
                              n_estimators=500, n_jobs=None,
                              num_parallel_tree=None, predictor=None,
                              random_state=None, ...))])

In [295]:
jb.dump(final_model,"Model_Flight_Ticket_Final.pkl")
jb.dump(X.columns,"Inputs_Flight_Ticket_Final.pkl")

['Inputs_Flight_Ticket_Final.pkl']

In [296]:
for col in df.columns:
    print(f"test_df.loc[0,'{col}']= {col}")

test_df.loc[0,'Airline']= Airline
test_df.loc[0,'Source']= Source
test_df.loc[0,'Destination']= Destination
test_df.loc[0,'Duration']= Duration
test_df.loc[0,'Additional_Info']= Additional_Info
test_df.loc[0,'Price']= Price
test_df.loc[0,'Total_Stops']= Total_Stops
test_df.loc[0,'Dep_Day']= Dep_Day
test_df.loc[0,'Dep_Month']= Dep_Month
test_df.loc[0,'Dep_Hour']= Dep_Hour
test_df.loc[0,'Dep_Minute']= Dep_Minute
test_df.loc[0,'Arrival_Day']= Arrival_Day
test_df.loc[0,'Arrival_Month']= Arrival_Month
test_df.loc[0,'Arrival_Hour']= Arrival_Hour
test_df.loc[0,'Arrival_Minute']= Arrival_Minute


In [297]:
df.columns

Index(['Airline', 'Source', 'Destination', 'Duration', 'Additional_Info',
       'Price', 'Total_Stops', 'Dep_Day', 'Dep_Month', 'Dep_Hour',
       'Dep_Minute', 'Arrival_Day', 'Arrival_Month', 'Arrival_Hour',
       'Arrival_Minute'],
      dtype='object')

In [298]:
## check value counts 
for col in df.columns:
    print (col)
    print(df[col].unique())
    print(50*'*')

Airline
['Air India' 'Jet Airways' 'IndiGo' 'SpiceJet' 'Multiple carriers' 'GoAir'
 'Vistara' 'Air Asia' 'Vistara Premium economy'
 'Multiple carriers Premium economy' 'Trujet']
**************************************************
Source
['Kolkata' 'Delhi' 'Banglore' 'Chennai' 'Mumbai']
**************************************************
Destination
['Banglore' 'Cochin' 'New Delhi' 'Kolkata' 'Delhi' 'Hyderabad']
**************************************************
Duration
[ 445 1140  325  285  145  930 1265 1530  470  795  155  135  730 1595
  270 1355 1380 1235  310  170  175  800  910  345  355  805 1320  330
  625  315  150  375  715  665  510 1325  165  720  965 1195  195 1520
  180  975  905  390 1505  745 1640  615  630   90   85  440  810  300
 1145  890  160 1330  575  600 1280 1125  740 1080  555 1050  995  735
  450 1440  535  430  870 1820  765  610  925  845 1390 1090  960  140
  480 1015  190  840 1430 1300 1275  650  495  515  710 1655  505 1255
  290  490 1465 1415 1545 1570

In [299]:
%%writefile streamlit_app_Flight_Ticket_final.py

import joblib
import streamlit as st
import category_encoders
import pandas as pd
import sklearn

Model= joblib.load("Model_Flight_Ticket_Final.pkl")
Inputs= joblib.load("Inputs_Flight_Ticket_Final.pkl")

def prediction(Airline, Source, Destination, Duration, Additional_Info,
       Total_Stops, Dep_Day, Dep_Month, Dep_Hour,
       Dep_Minute, Arrival_Day, Arrival_Month, Arrival_Hour,
       Arrival_Minute):
    test_df=pd.DataFrame(columns=Inputs)
    test_df.loc[0,'Airline']= Airline
    test_df.loc[0,'Source']= Source
    test_df.loc[0,'Destination']= Destination
    test_df.loc[0,'Duration']= Duration
    test_df.loc[0,'Additional_Info']= Additional_Info
    test_df.loc[0,'Total_Stops']= Total_Stops
    test_df.loc[0,'Dep_Day']= Dep_Day
    test_df.loc[0,'Dep_Month']= Dep_Month
    test_df.loc[0,'Dep_Hour']= Dep_Hour
    test_df.loc[0,'Dep_Minute']= Dep_Minute
    test_df.loc[0,'Arrival_Day']= Arrival_Day
    test_df.loc[0,'Arrival_Month']= Arrival_Month
    test_df.loc[0,'Arrival_Hour']= Arrival_Hour
    test_df.loc[0,'Arrival_Minute']= Arrival_Minute
    result= Model.predict(test_df)
    return result[0]

def main():    
    ## Setting up the page title
    st.set_page_config(page_title= 'Flight Tickets Price Prediction')
    
     # Add a title in the middle of the page using Markdown and CSS
    st.markdown("<h1 style='text-align: center;text-decoration: underline;color:GoldenRod'>Flight Tickets Price Prediction</h1>", unsafe_allow_html=True)
    
    Airline=st.selectbox('Insert Airline Company',  ['Air India', 'Jet Airways', 'IndiGo', 'SpiceJet', 'Multiple carriers', 'GoAir',
 'Vistara', 'Air Asia', 'Vistara Premium economy',
 'Multiple carriers Premium economy', 'Trujet'])
    
    Source=st.selectbox('Insert Departure City',['Kolkata', 'Delhi', 'Banglore', 'Chennai', 'Mumbai'])
    
    Destination=st.selectbox('Insert Arrival City',['Banglore', 'Cochin', 'New Delhi', 'Kolkata', 'Delhi', 'Hyderabad'])
    
    Duration=st.number_input('Insert duration of flight in minutes',min_value=30, max_value=2500, value=500,step=50)
    
    Additional_Info=st.selectbox('Insert Additional_Info',['No Info', 'In-flight meal not included', 'No check-in baggage included',
 '1 Long layover', 'Change airports', 'Red-eye flight'])

    Total_Stops=st.slider('Choose number of stops', min_value=0, max_value=4, value=2,step=1)
    
    Dep_Day=st.slider('Choose Depature Day', min_value=1, max_value=31, value=15,step=1)

    Dep_Month=st.slider('Choose Depature Month', min_value=1, max_value=12, value=6,step=1)

    Dep_Hour=st.slider('Choose Depature Hour', min_value=1, max_value=23, value=15,step=1)

    Dep_Minute=st.slider('Choose Depature Minute', min_value=1, max_value=59, value=30,step=1)

    Arrival_Day=st.slider('Choose Arrival Day', min_value=1, max_value=31, value=15,step=1)
    
    Arrival_Month=st.slider('Choose Arrival Month ', min_value=1, max_value=12, value=6,step=1)

    Arrival_Hour=st.slider('Choose Arrival Hour',min_value=1, max_value=23, value=15,step=1)

    Arrival_Minute=st.slider('Choose Arrival Minute', min_value=1, max_value=59, value=30,step=1)
    
    if st.button('predict'):
        results= prediction(Airline, Source, Destination, Duration, Additional_Info,Total_Stops, Dep_Day, Dep_Month, Dep_Hour, Dep_Minute, Arrival_Day, Arrival_Month, Arrival_Hour,Arrival_Minute)
        st.text(f"The Ticket Price is {int(results)} ")
main()


Overwriting streamlit_app_Flight_Ticket_final.py


In [300]:
!streamlit run streamlit_app_Flight_Ticket_final.py

^C
